In [10]:
# https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93

In [3]:
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

In [192]:
# Interval required 5 minutes
data = yf.download(tickers='SBRY.L', period='5d', interval='5m')
#Print data
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-03-16 08:00:00+00:00,236.600006,238.058594,236.600006,236.800003,236.800003,10446
2021-03-16 08:05:00+00:00,237.000000,237.800003,236.800003,237.360794,237.360794,68578
2021-03-16 08:10:00+00:00,237.699997,238.199997,237.539993,238.199997,238.199997,21036
2021-03-16 08:15:00+00:00,237.899994,238.399994,237.899994,238.100006,238.100006,7762
2021-03-16 08:20:00+00:00,238.500000,239.699997,238.500000,239.600006,239.600006,184342
...,...,...,...,...,...,...
2021-03-22 13:20:00+00:00,242.300003,242.395996,242.240494,242.240494,242.240494,2947
2021-03-22 13:25:00+00:00,242.395996,242.399994,242.199997,242.199997,242.199997,1412
2021-03-22 13:30:00+00:00,242.199997,242.600006,242.100006,242.399994,242.399994,29363


In [31]:
data = yf.download(tickers='sbry.l', period='1d', interval='5m')
#Print data
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-03-22 08:00:00+00:00,244.600006,244.600006,242.500000,243.199997,243.199997,6490
2021-03-22 08:05:00+00:00,243.199997,243.800003,243.000000,243.800003,243.800003,23763
2021-03-22 08:10:00+00:00,243.760803,244.399994,243.760803,244.199997,244.199997,5166
2021-03-22 08:15:00+00:00,244.300003,244.500000,244.199997,244.199997,244.199997,16282
2021-03-22 08:20:00+00:00,244.339996,244.800003,244.106003,244.800003,244.800003,10873
2021-03-22 08:25:00+00:00,244.800003,244.800003,244.199997,244.300003,244.300003,29345
2021-03-22 08:30:00+00:00,244.199997,244.399994,243.800003,244.100006,244.100006,7677
2021-03-22 08:35:00+00:00,244.199997,244.199997,244.100006,244.199997,244.199997,6877
2021-03-22 08:40:00+00:00,244.100006,244.140503,242.800003,242.800003,242.800003,290433


In [35]:
data = yf.download(tickers='AMTE.l', period='1y', interval='1h')
#Print data
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-03-12,250.000000,260.000000,220.000000,224.699997,224.699997,0
2021-03-12,220.000000,250.000000,220.000000,237.000000,237.000000,294495
2021-03-12,240.000000,245.000000,235.000000,235.050003,235.050003,64603
2021-03-12,233.750000,238.000000,232.050003,236.500015,236.500015,23009
2021-03-12,236.500015,237.000031,236.449966,237.000031,237.000031,18754
2021-03-12,236.699997,237.000000,234.000000,237.000000,237.000000,37330
2021-03-12,234.050003,234.699997,231.000000,231.000000,231.000000,33395
2021-03-12,231.000000,233.449997,231.000000,233.449997,233.449997,11308
2021-03-12,229.240005,235.000000,227.000000,233.500000,233.500000,51923


In [9]:
type(data)

pandas.core.frame.DataFrame

In [11]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/auto-complete"

querystring = {"q":"sbry","region":"Uk"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"explains":[],"count":10,"quotes":[{"exchange":"LSE","shortname":"SAINSBURY (J) PLC ORD 28 4/7P","quoteType":"EQUITY","symbol":"SBRY.L","index":"quotes","score":20346.0,"typeDisp":"Equity","longname":"J Sainsbury plc","isYahooFinance":true},{"exchange":"MEX","shortname":"SAINSBURY(J)","quoteType":"EQUITY","symbol":"SBRYN.MX","index":"quotes","score":20004.0,"typeDisp":"Equity","longname":"J Sainsbury plc","isYahooFinance":true}],"news":[{"uuid":"33e711cb-638f-3461-b7b8-03cbafadae9a","title":"U.K. Stocks Are at 20-Year Lows. Why It’s Time to Buy Post-Brexit.","publisher":"Barrons.com","link":"https://finance.yahoo.com/m/33e711cb-638f-3461-b7b8-03cbafadae9a/u.k.-stocks-are-at-20-year.html","providerPublishTime":1609925400,"type":"STORY"},{"uuid":"c6d1d65b-15be-3087-81ff-9a594fd48545","title":"Sterling surges above $1.35 on reports Brexit trade deal is ‘in sight’","publisher":"MarketWatch","link":"https://finance.yahoo.com/m/c6d1d65b-15be-3087-81ff-9a594fd48545/sterling-surges-above-%241

In [12]:
import http.client

conn = http.client.HTTPSConnection("apidojo-yahoo-finance-v1.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

conn.request("GET", "/auto-complete?q=sbry&region=UK", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"explains":[],"count":10,"quotes":[{"exchange":"LSE","shortname":"SAINSBURY (J) PLC ORD 28 4/7P","quoteType":"EQUITY","symbol":"SBRY.L","index":"quotes","score":20346.0,"typeDisp":"Equity","longname":"J Sainsbury plc","isYahooFinance":true},{"exchange":"MEX","shortname":"SAINSBURY(J)","quoteType":"EQUITY","symbol":"SBRYN.MX","index":"quotes","score":20004.0,"typeDisp":"Equity","longname":"J Sainsbury plc","isYahooFinance":true}],"news":[{"uuid":"33e711cb-638f-3461-b7b8-03cbafadae9a","title":"U.K. Stocks Are at 20-Year Lows. Why It’s Time to Buy Post-Brexit.","publisher":"Barrons.com","link":"https://finance.yahoo.com/m/33e711cb-638f-3461-b7b8-03cbafadae9a/u.k.-stocks-are-at-20-year.html","providerPublishTime":1609925400,"type":"STORY"},{"uuid":"c6d1d65b-15be-3087-81ff-9a594fd48545","title":"Sterling surges above $1.35 on reports Brexit trade deal is ‘in sight’","publisher":"MarketWatch","link":"https://finance.yahoo.com/m/c6d1d65b-15be-3087-81ff-9a594fd48545/sterling-surges-above-%241

In [13]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"

querystring = {"symbol":"SBRY","region":"UK"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [14]:
response

<Response [200]>

In [15]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

querystring = {"interval":"5m","symbol":"AMRN","range":"1d","region":"US"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"chart":{"result":[{"meta":{"currency":"USD","symbol":"AMRN","exchangeName":"NGM","instrumentType":"EQUITY","firstTradeDate":733674600,"regularMarketTime":1616184002,"gmtoffset":-14400,"timezone":"EDT","exchangeTimezoneName":"America/New_York","regularMarketPrice":6.44,"chartPreviousClose":6.36,"previousClose":6.36,"scale":3,"priceHint":2,"currentTradingPeriod":{"pre":{"timezone":"EDT","start":1616140800,"end":1616160600,"gmtoffset":-14400},"regular":{"timezone":"EDT","start":1616160600,"end":1616184000,"gmtoffset":-14400},"post":{"timezone":"EDT","start":1616184000,"end":1616198400,"gmtoffset":-14400}},"tradingPeriods":{"pre":[[{"timezone":"EDT","start":1616140800,"end":1616160600,"gmtoffset":-14400}]],"post":[[{"timezone":"EDT","start":1616184000,"end":1616198400,"gmtoffset":-14400}]],"regular":[[{"timezone":"EDT","start":1616160600,"end":1616184000,"gmtoffset":-14400}]]},"dataGranularity":"5m","range":"1d","validRanges":["1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"]

In [171]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

querystring = {"interval":"5m","symbol":"LLOY.L","range":"2w","region":"Uk"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"chart":{"result":[{"meta":{"currency":"GBp","symbol":"LLOY.L","exchangeName":"LSE","instrumentType":"EQUITY","firstTradeDate":820137600,"regularMarketTime":1616418119,"gmtoffset":0,"timezone":"GMT","exchangeTimezoneName":"Europe/London","regularMarketPrice":41.6,"chartPreviousClose":41.025,"previousClose":41.025,"scale":3,"priceHint":2,"currentTradingPeriod":{"pre":{"timezone":"GMT","start":1616397300,"end":1616400000,"gmtoffset":0},"regular":{"timezone":"GMT","start":1616400000,"end":1616430600,"gmtoffset":0},"post":{"timezone":"GMT","start":1616430600,"end":1616433300,"gmtoffset":0}},"tradingPeriods":{"pre":[[{"timezone":"GMT","start":1616397300,"end":1616400000,"gmtoffset":0}]],"post":[[{"timezone":"GMT","start":1616430600,"end":1616433300,"gmtoffset":0}]],"regular":[[{"timezone":"GMT","start":1616400000,"end":1616430600,"gmtoffset":0}]]},"dataGranularity":"5m","range":"1d","validRanges":["1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"]},"timestamp":[1616400000,161640

In [172]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

querystring = {"interval":"1h","symbol":"SBRY.L","range":"1m","region":"Uk"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"chart":{"result":[{"meta":{"currency":"GBp","symbol":"SBRY.L","exchangeName":"LSE","instrumentType":"EQUITY","firstTradeDate":583743600,"regularMarketTime":1616418104,"gmtoffset":0,"timezone":"GMT","exchangeTimezoneName":"Europe/London","regularMarketPrice":242.596,"chartPreviousClose":244.8,"previousClose":244.8,"scale":3,"priceHint":2,"currentTradingPeriod":{"pre":{"timezone":"GMT","start":1616397300,"end":1616400000,"gmtoffset":0},"regular":{"timezone":"GMT","start":1616400000,"end":1616430600,"gmtoffset":0},"post":{"timezone":"GMT","start":1616430600,"end":1616433300,"gmtoffset":0}},"tradingPeriods":{"pre":[[{"timezone":"GMT","start":1616397300,"end":1616400000,"gmtoffset":0}]],"post":[[{"timezone":"GMT","start":1616430600,"end":1616433300,"gmtoffset":0}]],"regular":[[{"timezone":"GMT","start":1616400000,"end":1616430600,"gmtoffset":0}]]},"dataGranularity":"5m","range":"1d","validRanges":["1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"]},"timestamp":[1616400000,16164

In [189]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"

querystring = {"interval":"1h","symbol":"AAPL","region":"US","range":"MAX"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"prices":[{"date":1616160600,"open":119.9000015258789,"high":121.43000030517578,"low":119.68000030517578,"close":119.98999786376953,"volume":185023200,"adjclose":119.98999786376953},{"date":1616074200,"open":122.87999725341797,"high":123.18000030517578,"low":120.31999969482422,"close":120.52999877929688,"volume":121229700,"adjclose":120.52999877929688},{"date":1615987800,"open":124.05000305175781,"high":125.86000061035156,"low":122.33999633789062,"close":124.76000213623047,"volume":111437500,"adjclose":124.76000213623047},{"date":1615901400,"open":125.69999694824219,"high":127.22000122070312,"low":124.72000122070312,"close":125.56999969482422,"volume":114740000,"adjclose":125.56999969482422},{"date":1615815000,"open":121.41000366210938,"high":124,"low":120.41999816894531,"close":123.98999786376953,"volume":92403800,"adjclose":123.98999786376953},{"date":1615559400,"open":120.4000015258789,"high":121.16999816894531,"low":119.16000366210938,"close":121.02999877929688,"volume":87963400,"

In [173]:
data = response.text

In [174]:
type(data)

str

In [175]:
print(len(data.split("}")))
# print(data.split("}")) # 用“}”将整个text变成包含15个parts的list

15


In [176]:
nine = data.split("}")
# seperate the big text in 15 segments with "}"，make it into a list

In [177]:
ten = nine[9].split(":") #截取timestamp - high整个这一大块儿
len(ten)

9

In [178]:
print(ten[0], ten[1]) 

,"timestamp" [1616400000,1616400300,1616400600,1616400900,1616401200,1616401500,1616401800,1616402100,1616402400,1616402700,1616403000,1616403300,1616403600,1616403900,1616404200,1616404500,1616404800,1616405100,1616405400,1616405700,1616406000,1616406300,1616406600,1616406900,1616407200,1616407500,1616407800,1616408100,1616408400,1616408700,1616409000,1616409300,1616409600,1616409900,1616410200,1616410500,1616410800,1616411100,1616411400,1616411700,1616412000,1616412300,1616412600,1616412900,1616413200,1616413500,1616413800,1616414100,1616414400,1616414700,1616415000,1616415300,1616415600,1616415900,1616416200,1616416500,1616416800,1616417100,1616417400,1616417700,1616418000,1616418104],"indicators"


In [179]:
eleven = ten[1].split(",")
print(eleven) # this is the timestamp's content

['[1616400000', '1616400300', '1616400600', '1616400900', '1616401200', '1616401500', '1616401800', '1616402100', '1616402400', '1616402700', '1616403000', '1616403300', '1616403600', '1616403900', '1616404200', '1616404500', '1616404800', '1616405100', '1616405400', '1616405700', '1616406000', '1616406300', '1616406600', '1616406900', '1616407200', '1616407500', '1616407800', '1616408100', '1616408400', '1616408700', '1616409000', '1616409300', '1616409600', '1616409900', '1616410200', '1616410500', '1616410800', '1616411100', '1616411400', '1616411700', '1616412000', '1616412300', '1616412600', '1616412900', '1616413200', '1616413500', '1616413800', '1616414100', '1616414400', '1616414700', '1616415000', '1616415300', '1616415600', '1616415900', '1616416200', '1616416500', '1616416800', '1616417100', '1616417400', '1616417700', '1616418000', '1616418104]', '"indicators"']


In [180]:
twelve = eleven[0:-1]
print(twelve)

['[1616400000', '1616400300', '1616400600', '1616400900', '1616401200', '1616401500', '1616401800', '1616402100', '1616402400', '1616402700', '1616403000', '1616403300', '1616403600', '1616403900', '1616404200', '1616404500', '1616404800', '1616405100', '1616405400', '1616405700', '1616406000', '1616406300', '1616406600', '1616406900', '1616407200', '1616407500', '1616407800', '1616408100', '1616408400', '1616408700', '1616409000', '1616409300', '1616409600', '1616409900', '1616410200', '1616410500', '1616410800', '1616411100', '1616411400', '1616411700', '1616412000', '1616412300', '1616412600', '1616412900', '1616413200', '1616413500', '1616413800', '1616414100', '1616414400', '1616414700', '1616415000', '1616415300', '1616415600', '1616415900', '1616416200', '1616416500', '1616416800', '1616417100', '1616417400', '1616417700', '1616418000', '1616418104]']


In [181]:
thirteen = []
for i in twelve:
    thirteen.append(i.replace(r"[","").replace(r"]","")) 
print(thirteen)

['1616400000', '1616400300', '1616400600', '1616400900', '1616401200', '1616401500', '1616401800', '1616402100', '1616402400', '1616402700', '1616403000', '1616403300', '1616403600', '1616403900', '1616404200', '1616404500', '1616404800', '1616405100', '1616405400', '1616405700', '1616406000', '1616406300', '1616406600', '1616406900', '1616407200', '1616407500', '1616407800', '1616408100', '1616408400', '1616408700', '1616409000', '1616409300', '1616409600', '1616409900', '1616410200', '1616410500', '1616410800', '1616411100', '1616411400', '1616411700', '1616412000', '1616412300', '1616412600', '1616412900', '1616413200', '1616413500', '1616413800', '1616414100', '1616414400', '1616414700', '1616415000', '1616415300', '1616415600', '1616415900', '1616416200', '1616416500', '1616416800', '1616417100', '1616417400', '1616417700', '1616418000', '1616418104']


In [182]:
timestamp = []
import datetime
for i in thirteen:
    timestamp.append(datetime.datetime.fromtimestamp(int(i)).isoformat())
    

In [183]:
print(timestamp)

['2021-03-22T08:00:00', '2021-03-22T08:05:00', '2021-03-22T08:10:00', '2021-03-22T08:15:00', '2021-03-22T08:20:00', '2021-03-22T08:25:00', '2021-03-22T08:30:00', '2021-03-22T08:35:00', '2021-03-22T08:40:00', '2021-03-22T08:45:00', '2021-03-22T08:50:00', '2021-03-22T08:55:00', '2021-03-22T09:00:00', '2021-03-22T09:05:00', '2021-03-22T09:10:00', '2021-03-22T09:15:00', '2021-03-22T09:20:00', '2021-03-22T09:25:00', '2021-03-22T09:30:00', '2021-03-22T09:35:00', '2021-03-22T09:40:00', '2021-03-22T09:45:00', '2021-03-22T09:50:00', '2021-03-22T09:55:00', '2021-03-22T10:00:00', '2021-03-22T10:05:00', '2021-03-22T10:10:00', '2021-03-22T10:15:00', '2021-03-22T10:20:00', '2021-03-22T10:25:00', '2021-03-22T10:30:00', '2021-03-22T10:35:00', '2021-03-22T10:40:00', '2021-03-22T10:45:00', '2021-03-22T10:50:00', '2021-03-22T10:55:00', '2021-03-22T11:00:00', '2021-03-22T11:05:00', '2021-03-22T11:10:00', '2021-03-22T11:15:00', '2021-03-22T11:20:00', '2021-03-22T11:25:00', '2021-03-22T11:30:00', '2021-03-2

In [190]:
datetime.datetime.fromtimestamp(1616160600).isoformat()

'2021-03-19T13:30:00'

In [191]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/auto-complete"

querystring = {"q":"tesla","region":"US"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"explains":[],"count":15,"quotes":[{"exchange":"NMS","shortname":"Tesla, Inc.","quoteType":"EQUITY","symbol":"TSLA","index":"quotes","score":450536.0,"typeDisp":"Equity","longname":"Tesla, Inc.","isYahooFinance":true},{"exchange":"GER","shortname":"TESLA INC. DL -,001","quoteType":"EQUITY","symbol":"TL0.DE","index":"quotes","score":20341.0,"typeDisp":"Equity","longname":"Tesla, Inc.","isYahooFinance":true},{"exchange":"FRA","shortname":"TESLA INC. DL -,001","quoteType":"EQUITY","symbol":"TL0.F","index":"quotes","score":20244.0,"typeDisp":"Equity","longname":"Tesla, Inc.","isYahooFinance":true},{"exchange":"LSE","shortname":"GRANITESHARES FINANCIAL PLC GRA","quoteType":"EQUITY","symbol":"3STS.L","index":"quotes","score":20180.0,"typeDisp":"Equity","longname":"GraniteShares 3x Short Tesla Daily ETP","isYahooFinance":true},{"exchange":"MEX","shortname":"TESLA INC","quoteType":"EQUITY","symbol":"TSLA.MX","index":"quotes","score":20164.0,"typeDisp":"Equity","longname":"Tesla, Inc.","isYaho